In [1]:
# Crear el notebook completo en un archivo .ipynb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Cargar datos
costos_df = pd.read_excel('Costos_Subprocesos.xlsx')
indicadores_df = pd.read_excel('Indicadores_Perforación.xlsx')

# Convertir las fechas a formato datetime
costos_df['Subproceso_Costo'] = pd.to_datetime(costos_df['Subproceso_Costo'])
indicadores_df['Mina/Rajo_Flota_Indicador'] = pd.to_datetime(indicadores_df['Mina/Rajo_Flota_Indicador'])

# Unir los DataFrames por la columna de fechas
df_unificado = pd.merge(costos_df, indicadores_df, left_on='Subproceso_Costo', right_on='Mina/Rajo_Flota_Indicador', how='inner')
df_unificado = df_unificado.drop(columns=['Mina/Rajo_Flota_Indicador'])

# Calcular la correlación entre todos los subprocesos y los indicadores
subprocesos_cols = costos_df.columns[1:]  # Excluyendo la columna de fecha
indicadores_cols = indicadores_df.columns[1:]  # Excluyendo la columna de fecha
correlation_matrix = df_unificado.corr()
correlation_filtered = correlation_matrix.loc[subprocesos_cols, indicadores_cols]
top_correlations = correlation_filtered.apply(lambda x: x.nlargest(3), axis=1)

# Dividir los datos en entrenamiento (2016-2021) y prueba (2022)
train_data = df_unificado[df_unificado['Subproceso_Costo'] < '2022-01-01']
test_data = df_unificado[df_unificado['Subproceso_Costo'] == '2022-01-01']

In [2]:
# Modelo 1: Linear Regression (Múltiple)
resultados_lr = {}
lr_model = LinearRegression()
imputer = SimpleImputer(strategy='mean')  # Imputar con la media

for subproceso in subprocesos_cols:
    indicadores_seleccionados = top_correlations.loc[subproceso].dropna().index.tolist()
    if len(indicadores_seleccionados) > 0:
        X_train = train_data[indicadores_seleccionados]
        y_train = train_data[subproceso]
        X_test = test_data[indicadores_seleccionados]
        
        # Imputar valores faltantes en X_train y X_test
        X_train = imputer.fit_transform(X_train)
        X_test = imputer.transform(X_test)
        
        # Entrenar el modelo
        lr_model.fit(X_train, y_train)
        
        # Realizar la predicción para el 2022
        y_pred_2022 = lr_model.predict(X_test)[0]
        costo_real_2022 = costos_df[costos_df['Subproceso_Costo'] == '2022-01-01'][subproceso].values[0]
        diferencia = y_pred_2022 - costo_real_2022
        diferencia_pct = (diferencia / costo_real_2022) * 100 if costo_real_2022 != 0 else 'N/A'
        resultados_lr[subproceso] = {'Predicción 2022': y_pred_2022, 'Costo Real 2022': costo_real_2022, 'Diferencia': diferencia, 'Diferencia %': diferencia_pct}

resultados_lr_df = pd.DataFrame(resultados_lr).T
print("Resultados para Linear Regression (Múltiple):")
print(resultados_lr_df)

Resultados para Linear Regression (Múltiple):
                                        Predicción 2022  Costo Real 2022  \
ADM OPERACIÓN Y GESTIÓN ESP              1695594.857562              0.0   
ADM OPERACIÓN Y GESTIÓN ESP SUR             8927.441737         30.27672   
ADM OPERACIÓN Y GESTIÓN MIR               160701.379119              0.0   
ADM OPERACIÓN Y GESTIÓN OXE              -464310.785967              0.0   
ADM OPERACIÓN Y GESTIÓN TC                170370.255256        906.14448   
ADM OPERACIÓN Y GESTIÓN TES SUR           116776.595251              0.0   
ADMINISTRACIÓN DESARROLLO MINA              1155.486889     881286.83064   
ADMINISTRACIÓN MINA                        -8620.083726    1780999.25816   
CARGUÍO                                 -1267324.063292     833630.60568   
CARGUÍO ENCUENTRO                           -15193.3529         262.9632   
CARGUÍO ESPERANZA                        2416230.982762              0.0   
CARGUÍO ESPERANZA SUR                     

In [3]:
# Modelo 2: Polynomial Regression (Regresión Polinómica)
resultados_pr = {}
for subproceso in subprocesos_cols:
    indicadores_seleccionados = top_correlations.loc[subproceso].dropna().index.tolist()
    if len(indicadores_seleccionados) > 0:
        X_train = train_data[indicadores_seleccionados]
        y_train = train_data[subproceso]
        X_test = test_data[indicadores_seleccionados]

        X_train = imputer.fit_transform(X_train)
        X_test = imputer.transform(X_test)
        
        poly = PolynomialFeatures(degree=2)
        X_train_poly = poly.fit_transform(X_train)
        X_test_poly = poly.transform(X_test)
        
        pr_model = LinearRegression()
        pr_model.fit(X_train_poly, y_train)
        y_pred_2022 = pr_model.predict(X_test_poly)[0]
        costo_real_2022 = costos_df[costos_df['Subproceso_Costo'] == '2022-01-01'][subproceso].values[0]
        diferencia = y_pred_2022 - costo_real_2022
        diferencia_pct = (diferencia / costo_real_2022) * 100 if costo_real_2022 != 0 else 'N/A'
        resultados_pr[subproceso] = {'Predicción 2022': y_pred_2022, 'Costo Real 2022': costo_real_2022, 'Diferencia': diferencia, 'Diferencia %': diferencia_pct}
resultados_pr_df = pd.DataFrame(resultados_pr).T
print("Resultados para Polynomial Regression (Regresión Polinómica):")
print(resultados_pr_df)

Resultados para Polynomial Regression (Regresión Polinómica):
                                        Predicción 2022  Costo Real 2022  \
ADM OPERACIÓN Y GESTIÓN ESP              1697587.861884              0.0   
ADM OPERACIÓN Y GESTIÓN ESP SUR             8927.441737         30.27672   
ADM OPERACIÓN Y GESTIÓN MIR              -212385.107717              0.0   
ADM OPERACIÓN Y GESTIÓN OXE               124481.000072              0.0   
ADM OPERACIÓN Y GESTIÓN TC                151904.044431        906.14448   
ADM OPERACIÓN Y GESTIÓN TES SUR           240401.098218              0.0   
ADMINISTRACIÓN DESARROLLO MINA             -2437.986474     881286.83064   
ADMINISTRACIÓN MINA                        37523.252883    1780999.25816   
CARGUÍO                                 -1246375.631874     833630.60568   
CARGUÍO ENCUENTRO                            -189.04909         262.9632   
CARGUÍO ESPERANZA                        1543453.978957              0.0   
CARGUÍO ESPERANZA SUR     

In [4]:
# Modelo 3: Neural Network (Redes Neuronales)
resultados_nn = {}
nn_model = MLPRegressor(hidden_layer_sizes=(100,), max_iter=1000, random_state=42)
for subproceso in subprocesos_cols:
    indicadores_seleccionados = top_correlations.loc[subproceso].dropna().index.tolist()
    if len(indicadores_seleccionados) > 0:
        X_train = train_data[indicadores_seleccionados]
        y_train = train_data[subproceso]
        X_test = test_data[indicadores_seleccionados]
        
        X_train = imputer.fit_transform(X_train)
        X_test = imputer.transform(X_test)
        
        nn_model.fit(X_train, y_train)
        y_pred_2022 = nn_model.predict(X_test)[0]
        costo_real_2022 = costos_df[costos_df['Subproceso_Costo'] == '2022-01-01'][subproceso].values[0]
        diferencia = y_pred_2022 - costo_real_2022
        diferencia_pct = (diferencia / costo_real_2022) * 100 if costo_real_2022 != 0 else 'N/A'
        resultados_nn[subproceso] = {'Predicción 2022': y_pred_2022, 'Costo Real 2022': costo_real_2022, 'Diferencia': diferencia, 'Diferencia %': diferencia_pct}
resultados_nn_df = pd.DataFrame(resultados_nn).T
print("Resultados para Neural Network (Redes Neuronales):")
print(resultados_nn_df)

C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum it

Resultados para Neural Network (Redes Neuronales):
                                       Predicción 2022  Costo Real 2022  \
ADM OPERACIÓN Y GESTIÓN ESP                 160.202352              0.0   
ADM OPERACIÓN Y GESTIÓN ESP SUR             104.906048         30.27672   
ADM OPERACIÓN Y GESTIÓN MIR              126763.123398              0.0   
ADM OPERACIÓN Y GESTIÓN OXE                 172.420239              0.0   
ADM OPERACIÓN Y GESTIÓN TC               170542.468506        906.14448   
ADM OPERACIÓN Y GESTIÓN TES SUR            57997.53773              0.0   
ADMINISTRACIÓN DESARROLLO MINA             1195.874203     881286.83064   
ADMINISTRACIÓN MINA                      -14224.768586    1780999.25816   
CARGUÍO                                     -76.585097     833630.60568   
CARGUÍO ENCUENTRO                            56.128029         262.9632   
CARGUÍO ESPERANZA                           347.736227              0.0   
CARGUÍO ESPERANZA SUR                       103.3

C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


In [5]:
# Modelo 4: Random Forest
resultados_rf = {}
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
for subproceso in subprocesos_cols:
    indicadores_seleccionados = top_correlations.loc[subproceso].dropna().index.tolist()
    if len(indicadores_seleccionados) > 0:
        X_train = train_data[indicadores_seleccionados]
        y_train = train_data[subproceso]
        X_test = test_data[indicadores_seleccionados]

        X_train = imputer.fit_transform(X_train)
        X_test = imputer.transform(X_test)
        
        rf_model.fit(X_train, y_train)
        y_pred_2022 = rf_model.predict(X_test)[0]
        costo_real_2022 = costos_df[costos_df['Subproceso_Costo'] == '2022-01-01'][subproceso].values[0]
        diferencia = y_pred_2022 - costo_real_2022
        diferencia_pct = (diferencia / costo_real_2022) * 100 if costo_real_2022 != 0 else 'N/A'
        resultados_rf[subproceso] = {'Predicción 2022': y_pred_2022, 'Costo Real 2022': costo_real_2022, 'Diferencia': diferencia, 'Diferencia %': diferencia_pct}
resultados_rf_df = pd.DataFrame(resultados_rf).T
print("Resultados para Random Forest:")
print(resultados_rf_df)

Resultados para Random Forest:
                                        Predicción 2022  Costo Real 2022  \
ADM OPERACIÓN Y GESTIÓN ESP              1685759.819836              0.0   
ADM OPERACIÓN Y GESTIÓN ESP SUR             8677.080201         30.27672   
ADM OPERACIÓN Y GESTIÓN MIR               102873.359933              0.0   
ADM OPERACIÓN Y GESTIÓN OXE                62603.649675              0.0   
ADM OPERACIÓN Y GESTIÓN TC                146882.249357        906.14448   
ADM OPERACIÓN Y GESTIÓN TES SUR           140206.573798              0.0   
ADMINISTRACIÓN DESARROLLO MINA              -312.705274     881286.83064   
ADMINISTRACIÓN MINA                        -2789.640699    1780999.25816   
CARGUÍO                                 -1235809.513256     833630.60568   
CARGUÍO ENCUENTRO                             77.167243         262.9632   
CARGUÍO ESPERANZA                        1991278.538857              0.0   
CARGUÍO ESPERANZA SUR                       1513.131645  

In [6]:
# Modelo 5: Gradient Boosting
resultados_gb = {}
gb_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
for subproceso in subprocesos_cols:
    indicadores_seleccionados = top_correlations.loc[subproceso].dropna().index.tolist()
    if len(indicadores_seleccionados) > 0:
        X_train = train_data[indicadores_seleccionados]
        y_train = train_data[subproceso]
        X_test = test_data[indicadores_seleccionados]

        X_train = imputer.fit_transform(X_train)
        X_test = imputer.transform(X_test)
        
        gb_model.fit(X_train, y_train)
        y_pred_2022 = gb_model.predict(X_test)[0]
        costo_real_2022 = costos_df[costos_df['Subproceso_Costo'] == '2022-01-01'][subproceso].values[0]
        diferencia = y_pred_2022 - costo_real_2022
        diferencia_pct = (diferencia / costo_real_2022) * 100 if costo_real_2022 != 0 else 'N/A'
        resultados_gb[subproceso] = {'Predicción 2022': y_pred_2022, 'Costo Real 2022': costo_real_2022, 'Diferencia': diferencia, 'Diferencia %': diferencia_pct}
resultados_gb_df = pd.DataFrame(resultados_gb).T
print("Resultados para Gradient Boosting:")
print(resultados_gb_df)

Resultados para Gradient Boosting:
                                        Predicción 2022  Costo Real 2022  \
ADM OPERACIÓN Y GESTIÓN ESP              1697469.166562              0.0   
ADM OPERACIÓN Y GESTIÓN ESP SUR             8927.441737         30.27672   
ADM OPERACIÓN Y GESTIÓN MIR               202161.792145              0.0   
ADM OPERACIÓN Y GESTIÓN OXE                57765.155777              0.0   
ADM OPERACIÓN Y GESTIÓN TC                134714.453758        906.14448   
ADM OPERACIÓN Y GESTIÓN TES SUR           147149.886603              0.0   
ADMINISTRACIÓN DESARROLLO MINA              -232.833855     881286.83064   
ADMINISTRACIÓN MINA                         -616.854245    1780999.25816   
CARGUÍO                                 -1245558.896975     833630.60568   
CARGUÍO ENCUENTRO                            -26.850857         262.9632   
CARGUÍO ESPERANZA                        2131997.628287              0.0   
CARGUÍO ESPERANZA SUR                       1592.6488

In [7]:
# Modelo 6: XGBoost
resultados_xgb = {}
xgb_model = XGBRegressor(n_estimators=100, random_state=42)
for subproceso in subprocesos_cols:
    indicadores_seleccionados = top_correlations.loc[subproceso].dropna().index.tolist()
    if len(indicadores_seleccionados) > 0:
        X_train = train_data[indicadores_seleccionados]
        y_train = train_data[subproceso]
        X_test = test_data[indicadores_seleccionados]

        X_train = imputer.fit_transform(X_train)
        X_test = imputer.transform(X_test)

        xgb_model.fit(X_train, y_train)
        y_pred_2022 = xgb_model.predict(X_test)[0]
        costo_real_2022 = costos_df[costos_df['Subproceso_Costo'] == '2022-01-01'][subproceso].values[0]
        diferencia = y_pred_2022 - costo_real_2022
        diferencia_pct = (diferencia / costo_real_2022) * 100 if costo_real_2022 != 0 else 'N/A'
        resultados_xgb[subproceso] = {'Predicción 2022': y_pred_2022, 'Costo Real 2022': costo_real_2022, 'Diferencia': diferencia, 'Diferencia %': diferencia_pct}
resultados_xgb_df = pd.DataFrame(resultados_xgb).T
print("Resultados para XGBoost:")
print(resultados_xgb_df)

Resultados para XGBoost:
                                       Predicción 2022  Costo Real 2022  \
ADM OPERACIÓN Y GESTIÓN ESP                 1697472.25              0.0   
ADM OPERACIÓN Y GESTIÓN ESP SUR            8927.441406         30.27672   
ADM OPERACIÓN Y GESTIÓN MIR               -3336.699951              0.0   
ADM OPERACIÓN Y GESTIÓN OXE               59947.613281              0.0   
ADM OPERACIÓN Y GESTIÓN TC               107907.359375        906.14448   
ADM OPERACIÓN Y GESTIÓN TES SUR           136675.46875              0.0   
ADMINISTRACIÓN DESARROLLO MINA             -234.681595     881286.83064   
ADMINISTRACIÓN MINA                        -586.695312    1780999.25816   
CARGUÍO                                     -1245149.0     833630.60568   
CARGUÍO ENCUENTRO                            79.181519         262.9632   
CARGUÍO ESPERANZA                            1664755.0              0.0   
CARGUÍO ESPERANZA SUR                      1592.648926              0.0   


In [8]:
# Comparación de resultados
resultados_totales = {
    'Linear Regression (Múltiple)': resultados_lr_df,
    'Polynomial Regression': resultados_pr_df,
    'Neural Network': resultados_nn_df,
    'Random Forest': resultados_rf_df,
    'Gradient Boosting': resultados_gb_df,
    'XGBoost': resultados_xgb_df
}
for modelo, resultados in resultados_totales.items():
    print(f"Resumen para {modelo}:")
    print(resultados[['Diferencia %']])
    print("\n\n")

Resumen para Linear Regression (Múltiple):
                                          Diferencia %
ADM OPERACIÓN Y GESTIÓN ESP                        N/A
ADM OPERACIÓN Y GESTIÓN ESP SUR           29386.158796
ADM OPERACIÓN Y GESTIÓN MIR                        N/A
ADM OPERACIÓN Y GESTIÓN OXE                        N/A
ADM OPERACIÓN Y GESTIÓN TC                 18701.66563
ADM OPERACIÓN Y GESTIÓN TES SUR                    N/A
ADMINISTRACIÓN DESARROLLO MINA              -99.868886
ADMINISTRACIÓN MINA                        -100.484003
CARGUÍO                                    -252.024656
CARGUÍO ENCUENTRO                         -5877.748712
CARGUÍO ESPERANZA                                  N/A
CARGUÍO ESPERANZA SUR                              N/A
CARGUÍO MIRADOR                                    N/A
CARGUÍO TESORO CENTRAL                      -28.210438
CARGUÍO TESORO SUR                                 N/A
EQUIPOS  AUXILIARES                        -153.728262
EQUIPOS DE SOPORTE (AP

In [9]:
import pandas as pd
import numpy as np

# Resultados de los modelos (simula los DataFrames que imprimiste anteriormente)
resultados = {
    'Linear Regression': resultados_lr_df,
    'Polynomial Regression': resultados_pr_df,
    'Neural Network': resultados_nn_df,
    'Random Forest': resultados_rf_df,
    'Gradient Boosting': resultados_gb_df,
    'XGBoost': resultados_xgb_df
}

# Crear un DataFrame para almacenar los mejores modelos para cada subproceso
mejor_modelo_df = pd.DataFrame(columns=['Subproceso', 'Mejor Modelo', 'Diferencia %'])

# Comparar los resultados para cada subproceso
for subproceso in resultados['Linear Regression'].index:
    mejor_modelo = None
    menor_diferencia = np.inf
    
    for modelo, df in resultados.items():
        if subproceso in df.index and df.loc[subproceso, 'Diferencia %'] != 'N/A':
            diferencia = abs(df.loc[subproceso, 'Diferencia %'])
            if diferencia < menor_diferencia:
                menor_diferencia = diferencia
                mejor_modelo = modelo
    
    # Guardar el mejor modelo para el subproceso
    nueva_fila = pd.DataFrame({
        'Subproceso': [subproceso],
        'Mejor Modelo': [mejor_modelo],
        'Diferencia %': [menor_diferencia]
    })
    
    mejor_modelo_df = pd.concat([mejor_modelo_df, nueva_fila], ignore_index=True)
    
# Exportar como Excel los resultados
mejor_modelo_df.to_excel('C:/Users/Administrator/Documents/GitHub/Hackathon2-2024/Desarollo/Resultados/Costos_Indicadores_Perforación_Imputacion.xlsx', index=False)

# Mostrar los resultados
mejor_modelo_df


C:\Users\Administrator\AppData\Local\Temp\ipykernel_6668\394072432.py:36: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mejor_modelo_df = pd.concat([mejor_modelo_df, nueva_fila], ignore_index=True)


,Subproceso,Mejor Modelo,Diferencia %
0,ADM OPERACIÓN Y GESTIÓN ESP,None,inf
1,ADM OPERACIÓN Y GESTIÓN ESP SUR,Neural Network,2.464908e+02
2,ADM OPERACIÓN Y GESTIÓN MIR,None,inf
3,ADM OPERACIÓN Y GESTIÓN OXE,None,inf
4,ADM OPERACIÓN Y GESTIÓN TC,XGBoost,1.180841e+04
5,ADM OPERACIÓN Y GESTIÓN TES SUR,None,inf
6,ADMINISTRACIÓN DESARROLLO MINA,Neural Network,9.986430e+01
7,ADMINISTRACIÓN MINA,Polynomial Regression,9.789313e+01
8,CARGUÍO,Neural Network,1.000092e+02
9,CARGUÍO ENCUENTRO,XGBoost,6.988875e+01
